# plot 3D delta wing
- this notebook plot the surface pressure of all the 32 cases
- only surface pressure map
- make 101 images for a smooth gif

In [ ]:
# Set the font family to Times New Roman
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 15

mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'

import numpy as np
import pandas as pd
import csv

import plotly
import plotly.graph_objs as go
import math
from plotly.offline import iplot
from scipy.interpolate import griddata

# define and plot the wing model with pressure map
# task = ['CP_nofilter', 'CP_filter']
task = 'CP_filter'


In [ ]:
L = 2500
U = 1
Uf = 1.5
chord_real = 0.3
T = 0.3 # s periodic of the gust = 0.3s
sampleRate = 1000
T_s = 1/sampleRate
t = np.linspace(0,T_s*L,L)
# t_star = t/T

t_star_orig = (np.linspace(0, L, L) - 500) / 1000 / 0.3
t_star = t_star_orig[::2]

case_num = np.arange(1, 33)
excluded_mask = ~np.isin(case_num, [13, 20, 25, 28])
case_num = case_num[excluded_mask]

# define NACA0012 airfoil coordinates

In [ ]:
# there are the positions of the 15 pressure ports, used for interpolation
# no p11, which is linked to stagnation point
x = [ 0.1, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.4, 0.5, 0.1, 0.3, 0.4, 0.5, 0.6, 0.7]
y = [ 0.2, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]

# load data

In [ ]:
#####################################################
# Suppress/hide the warning
np.seterr(invalid='ignore')
data_path = 'D:/myProject/Exam/Thesis/dashuai/sd7003/code/20230621_low_dim/deltaWing/'
CP_origin = pd.read_csv(data_path+'CP_plot_map.csv')
CP_origin.set_index('case_number', inplace=True)
if task == 'CP_nofilter':
    CP_show_dict = CP_origin.loc[:, CP_origin.columns.str.startswith('CP_nofilter')]
elif task == 'CP_filter':
    CP_show_dict = CP_origin.loc[:, CP_origin.columns.str.startswith('CP_filter')]
else:
    print('Error. Please define which task to run.')

CP_show = CP_show_dict.to_numpy()

In [ ]:
CP_origin.head()

In [ ]:
CP_show_dict.head()

# plot 3d cp map

## define plot func

In [ ]:
def plotWing(timestamp_show,xx,yy,vg,gridsz,pset, caseToPlot, show_figure=True, save_figure=False):
    # pset is to set the angle of the model in degree
    chord = 1
    grsize = gridsz 
    timestamp = (timestamp_show-500)/1000/0.3  # convert to dimensionless time
    timestamp = round(timestamp,1)
    
    if show_figure:
        print(timestamp)
    else:
        pass

    c_left  = np.linspace(0,1,int(grsize/2)).reshape(int(grsize/2),1)
    c_right = np.linspace(1,0,int(grsize/2)).reshape(int(grsize/2),1)
    c  = np.concatenate((c_left,c_right),axis=0)
    c  = c*chord
    zz = np.zeros((len(xx),len(xx)))

    # interpolation, to find the z values of naca0012, based on x and y
    for ii in range(0,grsize):
        zz[0:sum(1*(yy[:,ii]<=c[ii])),ii]=np.flipud(5*0.12*c[ii]*(
            0.2969*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(1/2)
            -0.1260*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(1)
            -0.3516*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(2)
            +0.2843*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(3)
            -0.1015*(yy[0:sum(1*(yy[:,ii]<=c[ii])),ii]/c[ii])**(4)))

    # remove excess
    zz[0,0] = 0

    for i in range(0,grsize):
        j = 0
        while zz[j,i] != 0:
            j = j+1
        if i == int(grsize/2) + 2 or i == int(grsize/2) + 1:
            xx[j+4:-1,i] = math.nan
            yy[j+4:-1,i] = math.nan
            zz[j+4:-1,i] = math.nan
        elif i == int(grsize/2) - 2 or i == int(grsize/2) - 1:
            xx[j+4:-1,i] = math.nan
            yy[j+4:-1,i] = math.nan
            zz[j+4:-1,i] = math.nan
        else:
            xx[j+3:-1,i] = math.nan
            yy[j+3:-1,i] = math.nan
            zz[j+3:-1,i] = math.nan

    lx = np.linspace(-0.3,0,grsize)
    ly = np.linspace(0,0.3,grsize)
    lxN = np.linspace(0,0.3,grsize)
    lyN = np.linspace(0.3,0,grsize)

    zzb = -zz
    zz  = zz  - 0.07*chord
    zzb = zzb - 0.07*chord

    # rotate translate
    yoffset = chord/2
    yy = yy - yoffset

    yy  = yy*math.cos(math.radians(pset)) - zz*math.sin(math.radians(pset))
    zz  = yy*math.sin(math.radians(pset)) + zz*math.cos(math.radians(pset))
    zzb = yy*math.sin(math.radians(pset)) + zzb*math.cos(math.radians(pset))

    yy = yy + yoffset
    yy = yy + 1.2*chord
    xx = xx + 0.02

    plotly.offline.init_notebook_mode()

    tmp = np.flipud(vg)
    vg  = np.fliplr(tmp)

    zz_up = np.nan_to_num(zz, nan=0)
    
    wing_model_up  = go.Surface(x=xx,y=yy,z=zz,showscale=False, surfacecolor=zz_up, colorscale='gray', opacity=1,
                                lighting=dict(ambient=0.5, diffuse=0, specular=0.5)
                                )      # upper section of the wing model

    # extract pressure map from zz --> upper surface
    zz_map = zz
    # Define the index of the three vertices of the triangle: the ports area (check the vg to find these indexes)
    x1, y1 = (int(grsize*0.2)+1, int(grsize*0.15))
    x2, y2 = (int(grsize*0.2)+1, int(grsize*0.45)-1)
    x3, y3 = (int(grsize*0.8)-1, int(grsize*0.45)-1) 
    # Create a boolean mask with the same shape as zz_map
    mask = np.zeros_like(zz_map, dtype=bool)
    # Iterate over the array and set all elements inside the triangle to True and outside to False
    for i in range(zz_map.shape[0]): # row
        for j in range(zz_map.shape[1]): # column
            if (i > min(x1,x2,x3) and i < max(x1,x2,x3)) and (j > min(y1,y2,y3) and j < max(y1,y2,y3)) and( i < (int(grsize*0.2)+1)+(((int(grsize*0.8)-1)-(int(grsize*0.2)+1))/((int(grsize*0.45)-1)-int(grsize*0.15)))*(j-int(grsize*0.15))):
                mask[i,j] = True
    zz_map[~mask] = np.nan

    zz_down = np.nan_to_num(zzb, nan=0)
    
    # down section of the wing model
    wing_model_down  = go.Surface(x=xx,y=yy,z=zzb,showscale=False, surfacecolor=zz_down, colorscale='gray', opacity=1,
                                  lighting=dict(ambient=0.9))     
    colorbar=dict(ticks='outside',  ticklen=3, tickwidth=3, tickcolor='black', 
                  tickfont=dict(family='Times New Roman',size=25,color='black'),
                  thickness=20, tickformat='.2f',
                  len=0.6, 
                  tickvals=np.arange(-3,2,1),   # (-4,0,0.5)
                  xanchor='left', x=0.99, yanchor='top', y=0.8)

    # because there is latex equation in the text, the font size does not work, instead, use \large, \huge to control the latex size
    annotations=[ # I don't know why latex doesn't work; Here use html symbol instead
                dict(x=0.02, y=0.9, xref="paper", yref="paper", text="$\huge t^*={}$".format(timestamp),  
                     # or use html, if latex does not work: '<i>t<sup>*</sub>=timestamp</i>',
                     font=dict(size=30, color='black'),showarrow=False),
                dict(x=1.2, y=0.9, xref="paper", yref="paper", text = r'$\huge C_P$',     # 1.05 0.86
                     # or use html, if latex does not work: text="<i>C<sub>P</sub></i>",
                     font=dict(size=20,color='black'),showarrow=False)
                ]
    #pressure map on the wing model
    wing_model_p_map = go.Surface(x=xx,y=yy,z=zz_map+0.01,surfacecolor=vg,colorscale='Jet',showscale=True,
                                  lighting=dict(ambient=0.9),
                                  cmin = -3, cmax = 1,   # -3 0.5
                                  colorbar = colorbar) 

    # plot the pressure ports on the wing surface
    x_port = [-0.086, # 0.0957=(0.66-0.086) / 6 = 0.0957      (0.676-0.086)=0.0983
              -0.086-0.0957*0, -0.086-0.0957*1, -0.086-0.0957*2,
              -0.086-0.0957*0, -0.086-0.0957*1, -0.086-0.0957*2, -0.086-0.0957*3, -0.086-0.0957*4,
              -0.086-0.0983*0,                  -0.086-0.0983*2, -0.086-0.0983*3, -0.086-0.0983*4, -0.086-0.0983*5, -0.086-0.0983*6]
    y_port = [2-0.2*0, # (2-1.4)/3=0.2
              2-0.2*1, 2-0.2*1, 2-0.2*1,
              2-0.2*2, 2-0.2*2, 2-0.2*2, 2-0.2*2, 2-0.2*2,
              2-0.2*3,          2-0.2*3, 2-0.2*3, 2-0.2*3, 2-0.2*3, 2-0.2*3]
    z_port_calib = 0.008
    z_port = [-0.01,
              -0.007+z_port_calib, -0.012+z_port_calib, -0.022+z_port_calib,
              -0.016+z_port_calib, -0.018+z_port_calib, -0.02+z_port_calib, -0.024+z_port_calib, -0.030+z_port_calib,
              -0.033,        -0.034, -0.035, -0.036, -0.038, -0.041]
    points = {}
    for i in range(len(x_port)):
        points[i] = [x_port[i],y_port[i],z_port[i]]
    points = list(points.values())

    # Add the interpolated points to the plot
    rect_colors = ['red', 'green', 'blue', 'purple', 'orange']
    # Create an empty list to store the interpolated points and rectangles trace
    scatter_points_rect = []
    for i, point in enumerate(points):
        scatter_points_rect.append(go.Scatter3d(x=[point[0]], y=[point[1]], z=[point[2]], 
                                                mode='markers',marker=dict(size=5, color='black', symbol='circle',
                                                line=dict(color='black', width=1)), showlegend=False))
    # plot the wing
    fig = go.Figure(data=[wing_model_up, wing_model_down, wing_model_p_map]
                    # +scatter_points_rect
                    )

    zoom_in_ration = 0.28  
    fig.update_layout(scene = {"xaxis": {"visible": False},
                               "yaxis": {"visible": False},
                               "zaxis": {"visible": False},
                               'camera_eye': {"x": -1*zoom_in_ration, "y": -2*zoom_in_ration, "z": 2*zoom_in_ration}, # best (-1 -2 2)
                               "aspectratio": {"x": 1, "y": 0.5, "z": 0.06}
                              },
                      scene_camera = {"center": {"x": -0.18-0.03, "y": 0, "z": -0.1}},      # -0.18 0 0
                      width=500, height=500,
                      annotations=annotations)
    if show_figure:
        iplot(fig)
    else:
        pass
    
    # save figure as svg
    if save_figure:
        if task == 'CP_nofilter':
            fig.write_image('./figure/deltawing_noFilter.svg', scale=2)
        elif task == 'CP_filter':
            path = './pressure/'
            fig.write_image(path+'case_'+str(caseToPlot)+'/'+str(caseToPlot)+'_'+str(timestamp_show)+'.png', scale=2)
        else:
            print('Error. Please define which task to run.')
    else:
        pass

print('plotWing funciton is done!')

## plot 3d wing, for reference cases
- np.linspace(290, 2000, 101); make 101 images so that the gif is smooth

In [ ]:
# plot the 3D model and the pressure contour
# port 16 links to stagnation point, so no use port 16
for iii in [13, 20, 25, 28]: # test cases: 13 20 25 28
    
    caseToPlot = iii     # for show: train cases 2 16 32 | test cases: 13 20 25 28
    print(f'Making 3D pressure map for case {caseToPlot}')
    
    data_P_temp = CP_show[(L*(caseToPlot-1)):(L*caseToPlot),0:15]

    # ########################

    # only plot ith point, for steady state 
    #(t*=-0.6,1,1.5,6, i = -0.6*0.3*1000+500=320, 1*0.3*1000+500=800, 1.5*0.3*1000+500=950, 6*0.3*1000+500=2300)
    # for after gust, t*_max = 6.66
    #(t*=6.5, i = 6.5*0.3*1000+500=2450)

    for i in np.linspace(290, 2000, 101): # [290, 800, 2000]:

        # i = 290   # 290 800 2000 --> -0.7, 1, 5
        i = int(i)

        p1 = data_P_temp[i-1,:]    # for reference

        chord = 1
        grSize = 450  # 500 or 1000 are better, but with much memory
        [xx,yy]=np.meshgrid(np.linspace(-1,1,grSize),np.linspace(0,1,grSize)) # mesh grid, a rectangle
        points = np.concatenate((np.array(x).reshape(15,1), np.array(y).reshape(15,1)), axis=1)
        vg = griddata(points,p1,(xx,yy),method='cubic')

        plotWing(i,xx,yy,vg,grSize,0,caseToPlot, show_figure=False, save_figure=True)

    print('Done!')

# clip the figures to make them smaller
- Microsoft Paint can be used to check the exact pixel values to clip

In [ ]:
from PIL import Image

def crop_image(input_path, output_path, crop_box):
    with Image.open(input_path) as img:
        cropped_img = img.crop(crop_box)
        cropped_img.save(output_path)
        
path = 'D:/myProject/Exam/Thesis/dashuai/sd7003/code/20230621_low_dim/deltaWing_another_plot/plot_wing_reconstruct_all_models/output_all_cases_only_pressure/'

for jjj in range(1,33):

    caseToPlot = jjj

    for i in [290, 800, 2000]:

        timestamp_show = i

        input_image_path = path+'case_'+str(caseToPlot)+'/'+str(caseToPlot)+'_'+str(timestamp_show)+'.png'
        output_image_path = path+'case_'+str(caseToPlot)+'/'+str(caseToPlot)+'_'+str(timestamp_show)+'_clip'+'.png'

        left = 135
        top = 230
        right = 775 
        bottom = 735

        crop_box = (left, top, right, bottom)
        crop_image(input_image_path, output_image_path, crop_box)

print('Done')